In [1]:
print("Ok")

Ok


In [2]:
%pwd

'/Users/tanushyarram/projects/med_chatbot/Medical-Chatbot-GenAI/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/tanushyarram/projects/med_chatbot/Medical-Chatbot-GenAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')


In [10]:
#extracted_data

In [8]:
#begin chunking the text
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [32]:
#text_chunks

In [10]:
# Install required packages
%pip install sentence-transformers langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    try:
        # Initialize embeddings with default model
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        return embeddings
    except ImportError as e:
        print("Please install required packages:")
        print("pip install sentence-transformers langchain-huggingface")
        raise e

# Usage
embeddings = download_hugging_face_embeddings()

Note: you may need to restart the kernel to use updated packages.


/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


In [61]:
from dotenv import load_dotenv
load_dotenv()

True

In [62]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [63]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [64]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"indexes":[{"name":"medicalbot","metric":"cosine","dimension":768,"status":{"ready":true,"state":"Ready"},"host":"medicalbot-rfvjr0h.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}]}'
INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"upsertedCount":32}'
DEBUG:pinecone.core.ope

In [65]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"indexes":[{"name":"medicalbot","metric":"cosine","dimension":768,"status":{"ready":true,"state":"Ready"},"host":"medicalbot-rfvjr0h.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}]}'
INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference


In [66]:
docsearch

In [67]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [68]:
retrieved_docs = retriever.invoke("What is Acne?")

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"results":[],"matches":[{"id":"6c21a0e6-4802-419b-96b4-b4aae912ac6e","score":0.649222553,"values":[],"metadata":{"page":37,"source":"Data/Medical_book.pdf","text":"Acidosis see Respiratory acidosis; Renal\\ntubular acidosis; Metabolic acidosis\\nAcne\\nDefinition\\nAcne is a common skin disease characterized by\\npimples on the face, chest, and back. It occurs when the\\npores of the skin become clogged with oil, dead skin\\ncells, and bacteria.\\nDescription\\nAcne vulgaris, the medical term for common acne, is\\nthe most common skin disease. It affects nearly 17 million\\npeople in the United States. While acne can arise at any"}},{"id":"494916a5-c97e-4034-aaae-e4a7c3d4f233","score":0.649204731,"values":[],"metadata":{"page":37,"source":"Data/Medical_book.pdf","text":"Acidosis see Respiratory acidosis; Renal\\ntubular acidosis; Metabolic acidosis\\nAcne\\nDefinition\\nAcne is a common skin disease characterized by\\npimples on

In [69]:
retrieved_docs

[Document(id='6c21a0e6-4802-419b-96b4-b4aae912ac6e', metadata={'page': 37.0, 'source': 'Data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='494916a5-c97e-4034-aaae-e4a7c3d4f233', metadata={'page': 37.0, 'source': 'Data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\n

In [70]:

from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [73]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"results":[],"matches":[{"id":"6c21a0e6-4802-419b-96b4-b4aae912ac6e","score":0.649222553,"values":[],"metadata":{"page":37,"source":"Data/Medical_book.pdf","text":"Acidosis see Respiratory acidosis; Renal\\ntubular acidosis; Metabolic acidosis\\nAcne\\nDefinition\\nAcne is a common skin disease characterized by\\npimples on the face, chest, and back. It occurs when the\\npores of the skin become clogged with oil, dead skin\\ncells, and bacteria.\\nDescription\\nAcne vulgaris, the medical term for common acne, is\\nthe most common skin disease. It affects nearly 17 million\\npeople in the United States. While acne can arise at any"}},{"id":"494916a5-c97e-4034-aaae-e4a7c3d4f233","score":0.649204731,"values":[],"metadata":{"page":37,"source":"Data/Medical_book.pdf","text":"Acidosis see Respiratory acidosis; Renal\\ntubular acidosis; Metabolic acidosis\\nAcne\\nDefinition\\nAcne is a common skin disease characterized by\\npimples on

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])